In [ ]:
pip install networkx numpy

In [ ]:
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching

# Build Graph first
import networkx as nx

def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G

## matrice de test :



def christofides_algorithm(G):
    # Trouver un arbre couvrant minimal
    T = nx.minimum_spanning_tree(G, weight='weight')

    # Trouver les sommets de degré impair dans l'arbre couvrant minimal
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]

    # Restreindre le graphe aux sommets de degré impair
    odd_graph = G.subgraph(odd_degree_nodes)

    # Trouver un matching parfait de poids minimum dans le graphe restreint
    matching = min_weight_matching(odd_graph)

    # Ajouter le matching à l'arbre couvrant minimal pour obtenir un multigraphe
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])

    # Trouver un circuit Eulerien dans le multigraphe
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    # Convertir le circuit Eulerien en un circuit Hamiltonien et calculer le coût
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    #print(eulerian_circuit)
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    # Ajouter le coût pour retourner au point de départ
    # Ajouter le coût pour retourner au point de départ pour former un cycle
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']

    return hamiltonian_circuit, total_cost

def tabu_search(G, initial_solution, initial_cost, num_iterations, initial_tabu_size, max_tabu_size):
    best_solution = initial_solution[:]
    best_cost = initial_cost
    tabu_list = []
    current_tabu_size = initial_tabu_size

    def adjust_tabu_size(iteration, best_cost_change):
        if best_cost_change == 0:
            return min(current_tabu_size + 1, max_tabu_size)
        return max(current_tabu_size - 1, initial_tabu_size)

    for iteration in range(num_iterations):
        # Create a new solution by swapping two random cities
        current_solution = best_solution[:]
        i, j = sorted(random.sample(range(len(best_solution)), 2))
        current_solution[i], current_solution[j] = current_solution[j], current_solution[i]

        # Calculate the total cost of the new solution
        current_cost = 0
        for idx in range(1, len(current_solution)):
            if (current_solution[idx-1], current_solution[idx]) in tabu_list:
                current_cost += G[current_solution[idx-1]][current_solution[idx]]['weight'] * 1.1  # Penalize moves in the tabu list by 10%
            else:
                current_cost += G[current_solution[idx-1]][current_solution[idx]]['weight']

        # Add cost of returning to the starting point to complete the cycle
        if (current_solution[-1], current_solution[0]) not in tabu_list:
            current_cost += G[current_solution[-1]][current_solution[0]]['weight']
        else:
            current_cost += G[current_solution[-1]][current_solution[0]]['weight'] * 1.1  # Penalize moves in the tabu list by 10%

        # If the new solution is better, update the best solution
        if current_cost < best_cost:
            best_solution, best_cost = current_solution, current_cost
            if (i, j) not in tabu_list:
                tabu_list.append((i, j))
                if len(tabu_list) > current_tabu_size:
                    tabu_list.pop(0)

        # Adjust the size of the tabu list based on recent changes in best cost
        current_tabu_size = adjust_tabu_size(iteration, best_cost - current_cost)

    return best_solution, best_cost



# Création d'un graphe complet avec des poids aléatoires pour les arêtes
G = build_graph_from_distance_matrix(matrix)
# Exécution de l'algorithme de Christofides
start_time = time.time()
initial_solution, initial_cost = christofides_algorithm(G)
christofides_time = time.time() - start_time
print(f"Christofides solution: {initial_solution}, Cost: {initial_cost}, Time: {christofides_time}s")

# Exécution de la recherche tabou
start_time = time.time()
best_solution, best_cost = tabu_search(G, initial_solution,initial_cost, 1000, 5, 10)
tabu_time = time.time() - start_time
print(f"Tabu Search best solution: {best_solution}, Cost: {best_cost}, Time: {tabu_time}s")




Christofides solution: [0, 1, 5, 9, 12, 11, 13, 14, 15, 10, 8, 6, 4, 2, 3, 7, 0], Cost: 255, Time: 0.004084110260009766s


TypeError: 'int' object is not subscriptable

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np

def load_xml_and_extract_matrix(xml_file_path):
    # Charger le fichier XML
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Trouver l'élément 'graph' qui contient les données de la matrice
    graph_element = root.find('graph')

    # Extraire la matrice symétrique
    return extract_symmetric_matrix(graph_element)

def extract_symmetric_matrix(graph_element):
    num_vertices = len(graph_element)  # Nombre de vertices (supposé carré)
    matrix = np.zeros((num_vertices, num_vertices), dtype=float)  # Initialiser la matrice de zéros

    # Parcourir chaque vertex
    for i, vertex in enumerate(graph_element):
        # Parcourir chaque edge du vertex
        for edge in vertex:
            j = int(edge.text) - 1  # Index du vertex connecté (base 1 à base 0)
            if i != j:  # Éviter de remplir la diagonale
                cost = float(edge.attrib['cost'])  # Coût de l'edge
                matrix[i, j] = cost
                matrix[j, i] = cost  # Assurer la symétrie

    return matrix

# Utiliser la fonction
file_path = '/content/gr24.xml'  # Remplacer par le chemin réel du fichier
matrice_distances = load_xml_and_extract_matrix(file_path)
print(matrice_distances)


[[  0. 187. 196. 228. 112. 196. 167. 154. 209.  86. 223. 191. 180.  83.
   50. 219.  74. 139.  53.  43. 128.  99. 228. 142.]
 [187.   0. 228. 158.  96.  88.  59.  63. 286. 124.  49. 121. 315. 172.
  232.  92.  81.  98. 138. 200.  76.  89. 235.  99.]
 [196. 228.   0.  96. 120.  77. 101. 105. 159. 156. 185.  27. 188. 149.
  264.  82. 182. 261. 239. 232. 146. 221. 108.  84.]
 [228. 158.  96.   0.  77.  63.  56.  34. 190.  40. 123.  83. 193.  79.
  148. 119. 105. 144. 123.  98.  32. 105. 119.  35.]
 [112.  96. 120.  77.   0.  56.  25.  29. 216. 124. 115.  47. 245. 139.
  232.  31. 150. 176. 207. 200.  76. 189. 165.  29.]
 [196.  88.  77.  63.  56.   0.  29.  22. 229.  95.  86.  64. 258. 134.
  203.  43. 121. 164. 178. 171.  47. 160. 178.  42.]
 [167.  59. 101.  56.  25.  29.   0. 229. 225.  82.  90.  68. 228. 112.
  190.  58. 108. 136. 165. 131.  30. 147. 154.  36.]
 [154.  63. 105.  34.  29.  22. 229.   0.  82. 207. 313. 173.  29. 126.
  248. 238. 310. 389. 367. 166. 222. 349.  71. 220.]


In [ ]:
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching


def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G
# Pour générer une solution initiale on pourais utiliser l'heuristique de Cristofid déjà implémenté
def christofides_algorithm(G):
    # Trouver un arbre couvrant minimal
    T = nx.minimum_spanning_tree(G, weight='weight')

    # Trouver les sommets de degré impair dans l'arbre couvrant minimal
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]

    # Restreindre le graphe aux sommets de degré impair
    odd_graph = G.subgraph(odd_degree_nodes)

    # Trouver un matching parfait de poids minimum dans le graphe restreint
    matching = min_weight_matching(odd_graph)

    # Ajouter le matching à l'arbre couvrant minimal pour obtenir un multigraphe
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])

    # Trouver un circuit Eulerien dans le multigraphe
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    # Convertir le circuit Eulerien en un circuit Hamiltonien et calculer le coût
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    #print(eulerian_circuit)
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    # Ajouter le coût pour retourner au point de départ
    # Ajouter le coût pour retourner au point de départ pour former un cycle
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']

    return hamiltonian_circuit, total_cost



# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

# Fonction pour générer une solution initiale
def generer_solution_initiale(distance_matrix):
    G=build_graph_from_distance_matrix(distance_matrix)
    hamiltonian_circuit, total_cost = christofides_algorithm(G)
    return hamiltonian_circuit
# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(solution):
    voisinage = []
    for i in range(1, len(solution) - 1):
        for j in range(i + 1, len(solution)):
            voisin = solution[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

# Implémentation de l'algorithme de recherche tabou
def recherche_tabou(matrice_distances, nb_iterations_max, taille_liste_tabou):

    # Initialisations
    solution_courante = generer_solution_initiale(matrice_distances)
    meilleure_solution = solution_courante[:]
    cout_meilleure_solution = calculer_cout(meilleure_solution, matrice_distances)
    liste_tabou = []

    # Boucle principale
    for iteration in range(nb_iterations_max):
        voisinage = generer_voisinage(solution_courante)
        couts_voisinage = [calculer_cout(voisin, matrice_distances) for voisin in voisinage]
        # Sélection du meilleur voisin non tabou ou qui respecte le critère d'aspiration
        voisinage_et_couts = sorted([(voisin, cout) for voisin, cout in zip(voisinage, couts_voisinage)], key=lambda x: x[1])
        for voisin, cout_voisin in voisinage_et_couts:
            if (tuple(voisin), cout_voisin) not in liste_tabou or cout_voisin < cout_meilleure_solution:
                solution_courante = voisin
                cout_solution_courante = cout_voisin
                break

        # Mise à jour de la liste tabou
        if len(liste_tabou) >= taille_liste_tabou:
            liste_tabou.pop(0)  # Supprime le plus ancien si la taille maximale est atteinte
        liste_tabou.append((tuple(solution_courante), cout_solution_courante))

        # Mise à jour de la meilleure solution
        if cout_solution_courante < cout_meilleure_solution:
            meilleure_solution = solution_courante[:]
            cout_meilleure_solution = cout_solution_courante

        print(f"Iteration {iteration}: Cout actuel = {cout_solution_courante}, Meilleur cout = {cout_meilleure_solution}")

    return meilleure_solution, cout_meilleure_solution

# Exemple d'utilisation de l'algorithme de recherche tabou pour le TSP
nb_villes = 16
# Création d'une matrice de distances aléatoire et symétrique avec des diagonales nulles
'''matrice_distances = [
[0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
[10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
[15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
[20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
[25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
[30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
[35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
[40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
[45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
[50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
[55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
[60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
[65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
[70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
[75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
[80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]'''

# Paramètres de l'algorithme
nb_iterations_max = 1000
taille_liste_tabou = 10

# Lancement de l'algorithme
meilleure_solution, cout_meilleure_solution = recherche_tabou(matrice_distances, nb_iterations_max, taille_liste_tabou)


Iteration 0: Cout actuel = 1998.0, Meilleur cout = 1998.0
Iteration 1: Cout actuel = 1768.0, Meilleur cout = 1768.0
Iteration 2: Cout actuel = 1624.0, Meilleur cout = 1624.0
Iteration 3: Cout actuel = 1631.0, Meilleur cout = 1624.0
Iteration 4: Cout actuel = 1662.0, Meilleur cout = 1624.0
Iteration 5: Cout actuel = 1655.0, Meilleur cout = 1624.0
Iteration 6: Cout actuel = 1714.0, Meilleur cout = 1624.0
Iteration 7: Cout actuel = 1603.0, Meilleur cout = 1603.0
Iteration 8: Cout actuel = 1610.0, Meilleur cout = 1603.0
Iteration 9: Cout actuel = 1657.0, Meilleur cout = 1603.0
Iteration 10: Cout actuel = 1525.0, Meilleur cout = 1525.0
Iteration 11: Cout actuel = 1631.0, Meilleur cout = 1525.0
Iteration 12: Cout actuel = 1639.0, Meilleur cout = 1525.0
Iteration 13: Cout actuel = 1546.0, Meilleur cout = 1525.0
Iteration 14: Cout actuel = 1577.0, Meilleur cout = 1525.0
Iteration 15: Cout actuel = 1636.0, Meilleur cout = 1525.0
Iteration 16: Cout actuel = 1707.0, Meilleur cout = 1525.0
Iterati

# Avec diversification et Intensification

In [41]:
import math
import time
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance(coordinates[i], coordinates[j])

    return distance_matrix

# Example usage
file_path = "/content/gil262.tsp"
coordinates = read_tsp_file(file_path)
matrice_distances = create_distance_matrix(coordinates)

In [20]:
import xml.etree.ElementTree as ET
import numpy as np

def load_xml_and_extract_matrix(xml_file_path):
    # Charger le fichier XML
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Trouver l'élément 'graph' qui contient les données de la matrice
    graph_element = root.find('graph')

    # Extraire la matrice symétrique
    return extract_symmetric_matrix(graph_element)

def extract_symmetric_matrix(graph_element):
    num_vertices = len(graph_element)  # Nombre de vertices (supposé carré)
    matrix = np.zeros((num_vertices, num_vertices), dtype=float)  # Initialiser la matrice de zéros

    # Parcourir chaque vertex
    for i, vertex in enumerate(graph_element):
        # Parcourir chaque edge du vertex
        for edge in vertex:
            j = int(edge.text) - 1  # Index du vertex connecté (base 1 à base 0)
            if i != j:  # Éviter de remplir la diagonale
                cost = float(edge.attrib['cost'])  # Coût de l'edge
                matrix[i, j] = cost
                matrix[j, i] = cost  # Assurer la symétrie

    return matrix

# Utiliser la fonction
file_path = '/content/gr17.xml'  # Remplacer par le chemin réel du fichier
matrice_distances = load_xml_and_extract_matrix(file_path)
print(matrice_distances)


[[  0. 257. 390. 661. 227. 488. 572. 530. 555. 289. 282. 638. 567. 466.
  420. 745. 518.]
 [257.   0. 661. 228. 169. 112. 196. 154. 372. 262. 110. 437. 191.  74.
   53. 472. 142.]
 [390. 661.   0. 169. 383. 120.  77. 105. 175. 476. 324. 240.  27. 182.
  239. 237.  84.]
 [661. 228. 169.   0. 120. 267. 351. 309. 338. 196.  61. 421. 346. 243.
  199. 528. 297.]
 [227. 169. 383. 120.   0. 351.  63.  34. 264. 360. 208. 329.  83. 105.
  123. 364.  35.]
 [488. 112. 120. 267. 351.   0.  34.  29. 232. 444. 292. 297.  47. 150.
  207. 332.  29.]
 [572. 196.  77. 351.  63.  34.   0. 232. 249. 402. 250. 314.  68. 108.
  165. 349.  36.]
 [530. 154. 105. 309.  34.  29. 232.   0. 402. 495. 352.  95. 189. 326.
  383. 202. 236.]
 [555. 372. 175. 338. 264. 232. 249. 402.   0. 352. 154. 578. 439. 336.
  240. 685. 390.]
 [289. 262. 476. 196. 360. 444. 402. 495. 352.   0. 578. 435. 287. 184.
  140. 542. 238.]
 [282. 110. 324.  61. 208. 292. 250. 352. 154. 578.   0. 287. 254. 391.
  448. 157. 301.]
 [638. 437

In [42]:
import numpy as np
import random
import time
# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

# Fonction pour générer une solution initiale
def generer_solution_initiale(nb_villes):
    solution = list(range(nb_villes))
    random.shuffle(solution)
    return solution

# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(solution):
    voisinage = []
    for i in range(1, len(solution) - 1):
        for j in range(i + 1, len(solution)):
            voisin = solution[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

# Implémentation de l'algorithme de recherche tabou
def recherche_tabou(matrice_distances, nb_iterations_max, taille_liste_tabou):
    nb_villes = len(matrice_distances)

    # Initialisations
    solution_courante = generer_solution_initiale(nb_villes)
    meilleure_solution = solution_courante[:]
    cout_meilleure_solution = calculer_cout(meilleure_solution, matrice_distances)
    liste_tabou = []

    # Boucle principale
    for iteration in range(nb_iterations_max):
        voisinage = generer_voisinage(solution_courante)
        couts_voisinage = [calculer_cout(voisin, matrice_distances) for voisin in voisinage]
        # Sélection du meilleur voisin non tabou ou qui respecte le critère d'aspiration
        voisinage_et_couts = sorted([(voisin, cout) for voisin, cout in zip(voisinage, couts_voisinage)], key=lambda x: x[1])
        for voisin, cout_voisin in voisinage_et_couts:
            if (tuple(voisin), cout_voisin) not in liste_tabou or cout_voisin < cout_meilleure_solution:
                solution_courante = voisin
                cout_solution_courante = cout_voisin
                break

        # Mise à jour de la liste tabou
        if len(liste_tabou) >= taille_liste_tabou:
            liste_tabou.pop(0)  # Supprime le plus ancien si la taille maximale est atteinte
        liste_tabou.append((tuple(solution_courante), cout_solution_courante))

        # Mise à jour de la meilleure solution
        if cout_solution_courante < cout_meilleure_solution:
            meilleure_solution = solution_courante[:]
            cout_meilleure_solution = cout_solution_courante

        print(f"Iteration {iteration}: Cout actuel = {cout_solution_courante}, Meilleur cout = {cout_meilleure_solution}")

    return meilleure_solution, cout_meilleure_solution

# Exemple d'utilisation de l'algorithme de recherche tabou pour le TSP
nb_villes = 101
# Création d'une matrice de distances aléatoire et symétrique avec des diagonales nulles
'''matrice_distances = [
[0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
[10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
[15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
[20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
[25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
[30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
[35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
[40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
[45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
[50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
[55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
[60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
[65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
[70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
[75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
[80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]'''

# Paramètres de l'algorithme
nb_iterations_max = 500
taille_liste_tabou = 15
start_time = time.time()
# Lancement de l'algorithme
meilleure_solution, cout_meilleure_solution = recherche_tabou(matrice_distances, nb_iterations_max, taille_liste_tabou)


end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed Time: {elapsed_time} seconds")

Iteration 0: Cout actuel = 26383.431599026764, Meilleur cout = 26383.431599026764
Iteration 1: Cout actuel = 26026.05613605027, Meilleur cout = 26026.05613605027
Iteration 2: Cout actuel = 25672.49563446058, Meilleur cout = 25672.49563446058
Iteration 3: Cout actuel = 25323.545611044654, Meilleur cout = 25323.545611044654
Iteration 4: Cout actuel = 24986.164721197536, Meilleur cout = 24986.164721197536
Iteration 5: Cout actuel = 24660.32356914181, Meilleur cout = 24660.32356914181
Iteration 6: Cout actuel = 24355.945780168957, Meilleur cout = 24355.945780168957
Iteration 7: Cout actuel = 24051.539555969706, Meilleur cout = 24051.539555969706
Iteration 8: Cout actuel = 23748.9442178186, Meilleur cout = 23748.9442178186
Iteration 9: Cout actuel = 23447.327756451, Meilleur cout = 23447.327756451
Iteration 10: Cout actuel = 23146.878970116035, Meilleur cout = 23146.878970116035
Iteration 11: Cout actuel = 22852.84234051185, Meilleur cout = 22852.84234051185
Iteration 12: Cout actuel = 2256

KeyboardInterrupt: 

In [11]:
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching



def christofides_algorithm(G):
    # Trouver un arbre couvrant minimal
    T = nx.minimum_spanning_tree(G, weight='weight')

    # Trouver les sommets de degré impair dans l'arbre couvrant minimal
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]

    # Restreindre le graphe aux sommets de degré impair
    odd_graph = G.subgraph(odd_degree_nodes)

    # Trouver un matching parfait de poids minimum dans le graphe restreint
    matching = min_weight_matching(odd_graph)

    # Ajouter le matching à l'arbre couvrant minimal pour obtenir un multigraphe
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])

    # Trouver un circuit Eulerien dans le multigraphe
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    # Convertir le circuit Eulerien en un circuit Hamiltonien et calculer le coût
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    #print(eulerian_circuit)
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    # Ajouter le coût pour retourner au point de départ
    # Ajouter le coût pour retourner au point de départ pour former un cycle
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']

    return hamiltonian_circuit, total_cost
def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G
# Vous devriez avoir les fonctions calculer_cout, generer_solution_initiale, generer_voisinage, christofides_algorithm déjà définies comme dans votre code précédent.
def calculate_distance(city1, city2, adjacency_matrix):
    """Calculates the distance between two cities using the adjacency matrix."""
    if isinstance(city1, int) and isinstance(city2, int):
        return adjacency_matrix[city1][city2]
    else:
        raise ValueError("City indices must be integers")


def recherche_locale_2opt(adjacency_matrix, route):
    """Performs the 2-opt algorithm to improve a given route."""
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route) - 1):  # Assure that we have at least one element between i and j
                # Calculate the current distances between i and i+1, and j and j+1
                current_distance = calculate_distance(route[i], route[i + 1], adjacency_matrix) + \
                                   calculate_distance(route[j], route[j + 1], adjacency_matrix)

                # Calculate what the distance would be if we swapped i+1 and j
                new_distance = calculate_distance(route[i], route[j], adjacency_matrix) + \
                               calculate_distance(route[i + 1], route[j + 1], adjacency_matrix)

                # If the new distance is less, perform the swap and mark that we improved the route
                if new_distance < current_distance:
                    # Perform the swap: reverse the segment between i+1 and j
                    route[i + 1:j + 1] = route[i + 1:j + 1][::-1]
                    improved = True

    return route



def calculer_cout(route, matrice_distances):
    """Calculates the total travel cost of a given route using the distance matrix."""
    cout_total = 0
    for i in range(len(route) - 1):
        # Get the distance between the current city and the next city
        distance = matrice_distances[route[i]][route[i + 1]]
        cout_total += distance
    # Add the distance to return to the starting city
    cout_total += matrice_distances[route[-1]][route[0]]
    return cout_total


# Fonction pour générer une solution initiale
def generer_solution_initiale(nb_villes):
    # Crée une liste des villes à partir de 1 à nb_villes-1 car 0 est déjà inclus comme point de départ
    villes = list(range(0, nb_villes))
    random.shuffle(villes)
    # Ajoute 0 au début et à la fin pour fermer le circuit
    return [0] + villes + [0]

# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S)):
        for j in range(i, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

def hybrid_tabu_search(G, nb_villes, matrice_distances, nb_iterations_max, taille_liste_tabou, max_iter_sans_amelioration, seuil_variance):
    meilleure_solution, meilleur_cout = christofides_algorithm(G)
    #meilleure_solution = recherche_locale_2opt(matrice_distances, meilleure_solution)

    liste_tabou = []
    iter_sans_amelioration = 0
    var_couts = []

    for iteration in range(nb_iterations_max):
        if iter_sans_amelioration >= max_iter_sans_amelioration:  # Critère d'arrêt
            break

        voisinage = generer_voisinage(meilleure_solution)
        for voisin in voisinage:
            cout_voisin = calculer_cout(voisin, matrice_distances)
            if cout_voisin < meilleur_cout and (voisin not in liste_tabou):
                meilleure_solution = voisin
                meilleur_cout = cout_voisin
                iter_sans_amelioration = 0  # Réinitialiser le compteur
                var_couts.append(meilleur_cout)
                break  # Sortie précoce pour intensification
        else:
            iter_sans_amelioration += 1  # Incrémente si aucune amélioration n'est trouvée dans ce cycle

        # Diversification et Intensification
        if iteration % 100 == 0 and iteration != 0:
            if random.random() < 0.5:  # Diversification
                meilleure_solution = generer_solution_initiale(nb_villes)
                meilleur_cout = calculer_cout(meilleure_solution, matrice_distances)
            else:  # Intensification
                meilleure_solution = recherche_locale_2opt(meilleure_solution, matrice_distances)
                meilleur_cout = calculer_cout(meilleure_solution, matrice_distances)


        liste_tabou.append(meilleure_solution.copy())
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)

        if len(var_couts) > 10:
            variance = np.var(var_couts[-10:])
            if variance < seuil_variance:
                break

    return meilleure_solution, meilleur_cout


nb_villes = 53
# Création du graphe
G = build_graph_from_distance_matrix(matrice_distances)


# Paramètres pour la recherche taboue
nb_iterations_max = 100
taille_liste_tabou = 15
max_iter_sans_amelioration = 20
seuil_variance = 1e-4

# Exécution de l'algorithme hybride
solution, cout = hybrid_tabu_search(G, nb_villes, matrice_distances, nb_iterations_max, taille_liste_tabou, max_iter_sans_amelioration, seuil_variance)

print("Solution finale:", solution)
print("Coût de la solution:", calculer_cout(solution, matrice_distances))



Solution finale: [0, 39, 43, 42, 45, 16, 22, 2, 29, 17, 24, 1, 3, 30, 8, 4, 12, 9, 10, 11, 15, 26, 19, 31, 25, 20, 7, 13, 23, 40, 32, 6, 21, 33, 38, 46, 35, 14, 28, 5, 27, 47, 34, 18, 37, 41, 44, 36, 0]
Coût de la solution: 5273.0


In [32]:
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching




def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G
# Vous devriez avoir les fonctions calculer_cout, generer_solution_initiale, generer_voisinage, christofides_algorithm déjà définies comme dans votre code précédent.
def calculate_distance(city1, city2, adjacency_matrix):
    """Calculates the distance between two cities using the adjacency matrix."""
    if isinstance(city1, int) and isinstance(city2, int):
        return adjacency_matrix[city1][city2]


def recherche_locale_2opt(adjacency_matrix, route):
    """Performs the 2-opt algorithm to improve a given route. Assumes route and adjacency_matrix are properly formatted."""
    # Ensure that all route indices are integers
    route = [int(r) for r in route]

    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route) - 1):  # Assure that we have at least one element between i and j
                # Calculate the current distances between i and i+1, and j and j+1
                current_distance = adjacency_matrix[route[i]][route[i + 1]] + \
                                   adjacency_matrix[route[j]][route[j + 1]]

                # Calculate what the distance would be if we swapped i+1 and j
                new_distance = adjacency_matrix[route[i]][route[j]] + \
                               adjacency_matrix[route[i + 1]][route[j + 1]]

                # If the new distance is less, perform the swap and mark that we improved the route
                if new_distance < current_distance:
                    # Perform the swap: reverse the segment between i+1 and j
                    route[i + 1:j + 1] = route[i + 1:j + 1][::-1]
                    improved = True

    return route



def calculer_cout(route, matrice_distances):
    """Calculates the total travel cost of a given route using the distance matrix."""
    cout_total = 0
    for i in range(len(route) - 1):
        # Get the distance between the current city and the next city
        distance = matrice_distances[route[i]][route[i + 1]]
        cout_total += distance
    # Add the distance to return to the starting city
    cout_total += matrice_distances[route[-1]][route[0]]
    return cout_total


# Fonction pour générer une solution initiale
def generer_solution_initiale(nb_villes):
    # Crée une liste des villes à partir de 1 à nb_villes-1 car 0 est déjà inclus comme point de départ
    villes = list(range(0, nb_villes))
    random.shuffle(villes)
    # Ajoute 0 au début et à la fin pour fermer le circuit
    return  villes

# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S)):
        for j in range(i, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage





def christofides_algorithm(G):
    # Trouver un arbre couvrant minimal
    T = nx.minimum_spanning_tree(G, weight='weight')

    # Trouver les sommets de degré impair dans l'arbre couvrant minimal
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]

    # Restreindre le graphe aux sommets de degré impair
    odd_graph = G.subgraph(odd_degree_nodes)

    # Trouver un matching parfait de poids minimum dans le graphe restreint
    matching = min_weight_matching(odd_graph)

    # Ajouter le matching à l'arbre couvrant minimal pour obtenir un multigraphe
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])

    # Trouver un circuit Eulerien dans le multigraphe
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    # Convertir le circuit Eulerien en un circuit Hamiltonien et calculer le coût
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    #print(eulerian_circuit)
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    # Ajouter le coût pour retourner au point de départ
    # Ajouter le coût pour retourner au point de départ pour former un cycle
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']

    return hamiltonian_circuit, total_cost


# Fonction pour générer une solution initiale
def generer_solution_initiale(nb_villes):
    return random.sample(range(nb_villes), nb_villes)



# Algorithme de recherche tabou avec intensification, diversification et critère d'aspiration
def recherche_tabou(G,nb_villes, nb_iterations_max, taille_liste_tabou, matrice_distances, diversifier, intensifier, stop_stagnation, max_iter_sans_amelioration, initialize_randomly,seuil_variance):
    # Initialisation
    if initialize_randomly:
      S = generer_solution_initiale(nb_villes)
      meilleur_cout = calculer_cout(S, matrice_distances)
    else:
     S, meilleur_cout = christofides_algorithm(G) # Christofides
     S = recherche_locale_2opt(adjacency_matrix=matrice_distances ,route=S)

    meilleure_solution = S[:]
    liste_tabou = []
    iter_sans_amelioration = 0
    var_couts = []

    # Boucle principale
    for iteration in range(nb_iterations_max):
        voisinage = generer_voisinage(S)
        S_prime, cout_S_prime = None, float('inf')

        # Recherche d'une meilleure solution dans le voisinage
        for voisin in voisinage:
            cout_voisin = calculer_cout(voisin, matrice_distances)
            if (cout_voisin < cout_S_prime and
                (voisin not in liste_tabou or cout_voisin < meilleur_cout)):
                S_prime = voisin
                cout_S_prime = cout_voisin


        # Mise à jour de la meilleure solution trouvée
        if S_prime and cout_S_prime < meilleur_cout:
            meilleure_solution = S_prime[:]
            meilleur_cout = cout_S_prime

            if stop_stagnation:
              #print(iter_sans_amelioration)
              iter_sans_amelioration = 0  # Réinitialisation du compteur d'itérations sans amélioration
            var_couts.append(cout_S_prime)
        else:
            if stop_stagnation:
              iter_sans_amelioration += 1  # Incrémenter le compteur

        # Mise à jour de la solution actuelle et de la liste tabou
        S = S_prime or S
        liste_tabou.append(S)
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)

        # Intensification: Revenir à la meilleure solution périodiquement
       # Diversification et Intensification
        if iteration % 100 == 0 and iteration != 0:
            if (random.random() > 0.5):  # Diversification
              if diversifier:
                S= generer_solution_initiale(nb_villes)
            else:  # Intensification
               if intensifier:
                 if (random.random() > 0.5):
                    print(S_prime)
                    S = recherche_locale_2opt(adjacency_matrix=matrice_distances ,route=S_prime)
                 else :
                    S = generer_solution_initiale(nb_villes)
                    liste_tabou = []


        # Stop si stagnation
        if stop_stagnation:
          if iter_sans_amelioration >= max_iter_sans_amelioration:
              print(f"Stagnation atteinte à l'itération {iteration}, arrêt prématuré.")
              break

        # Variation de la taille de la liste tabou
        taille_liste_tabou = 10 if iteration % 15 == 0 else 15

        print(f"Iteration {iteration}: Cout actuel = {cout_S_prime}, Meilleur cout = {meilleur_cout}")

        if len(var_couts) > 10:
            variance = np.var(var_couts[-10:])
            if variance < seuil_variance:
                break

    return meilleure_solution, meilleur_cout
G=build_graph_from_distance_matrix(matrice_distances)
# Paramètres de l'algorithme
nb_villes = 262 # Nombre de villes
nb_iterations_max = 1000  # Nombre d'itérations maximal
taille_liste_tabou_initiale = 20  # Taille initiale de la liste tabou
diversifier = True
intensifier = True
stop_stagnation = True
max_iter_sans_amelioration =30
initialize_randomly = False
seuil_variance = 1e-4
start_time = time.time()
# Exécution de l'algorithme de recherche tabou
meilleure_solution, cout_meilleure_solution = recherche_tabou(G,nb_villes, nb_iterations_max, taille_liste_tabou_initiale, matrice_distances, diversifier, intensifier, stop_stagnation, 30, initialize_randomly,seuil_variance)



end_time = time.time()
elapsed_time = end_time - start_time
print(cout_meilleure_solution)

print(f"Elapsed Time: {elapsed_time} seconds")



Iteration 0: Cout actuel = 2481.998156924036, Meilleur cout = 2481.998156924036
Iteration 1: Cout actuel = 2481.9981569240376, Meilleur cout = 2481.998156924036
Iteration 2: Cout actuel = 2482.0341914069318, Meilleur cout = 2481.998156924036
Iteration 3: Cout actuel = 2481.580695961236, Meilleur cout = 2481.580695961236
Iteration 4: Cout actuel = 2481.5806959612346, Meilleur cout = 2481.5806959612346
Iteration 5: Cout actuel = 2481.635763998886, Meilleur cout = 2481.5806959612346
Iteration 6: Cout actuel = 2472.150763110309, Meilleur cout = 2472.150763110309
Iteration 7: Cout actuel = 2472.150763110311, Meilleur cout = 2472.150763110309
Iteration 8: Cout actuel = 2472.304673951004, Meilleur cout = 2472.150763110309
Iteration 9: Cout actuel = 2472.304673951003, Meilleur cout = 2472.150763110309
Iteration 10: Cout actuel = 2472.52416324401, Meilleur cout = 2472.150763110309
Iteration 11: Cout actuel = 2472.3702524033165, Meilleur cout = 2472.150763110309
Iteration 12: Cout actuel = 2472.

In [ ]:
import networkx as nx
import numpy as np
import random
import time
from networkx.algorithms.matching import min_weight_matching

def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G
def christofides_algorithm(G):
    # Trouver un arbre couvrant minimal
    T = nx.minimum_spanning_tree(G, weight='weight')

    # Trouver les sommets de degré impair dans l'arbre couvrant minimal
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]

    # Restreindre le graphe aux sommets de degré impair
    odd_graph = G.subgraph(odd_degree_nodes)

    # Trouver un matching parfait de poids minimum dans le graphe restreint
    matching = min_weight_matching(odd_graph)

    # Ajouter le matching à l'arbre couvrant minimal pour obtenir un multigraphe
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])

    # Trouver un circuit Eulerien dans le multigraphe
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    # Convertir le circuit Eulerien en un circuit Hamiltonien et calculer le coût
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    #print(eulerian_circuit)
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    # Ajouter le coût pour retourner au point de départ
    # Ajouter le coût pour retourner au point de départ pour former un cycle
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']

    return hamiltonian_circuit, total_cost
# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

# Fonction pour générer une solution initiale
def generer_solution_initiale(nb_villes):
    return random.sample(range(nb_villes), nb_villes)

# Fonction pour générer un voisinage (2-opt swap)
def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S) - 1):
        for j in range(i + 1, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

# Algorithme de recherche tabou avec intensification, diversification et critère d'aspiration
def recherche_tabou(G,nb_villes, nb_iterations_max, taille_liste_tabou, matrice_distances, diversifier, intensifier, stop_stagnation, max_iter_sans_amelioration, initialize_randomly):
    # Initialisation
    if initialize_randomly:
      S = generer_solution_initiale(nb_villes)
      meilleur_cout = calculer_cout(S, matrice_distances)
    else:
     S, meilleur_cout = christofides_algorithm(G) # Christofides
    meilleure_solution = S[:]
    liste_tabou = []
    iter_sans_amelioration = 0

    # Boucle principale
    for iteration in range(nb_iterations_max):
        voisinage = generer_voisinage(S)
        S_prime, cout_S_prime = None, float('inf')

        # Recherche d'une meilleure solution dans le voisinage
        for voisin in voisinage:
            cout_voisin = calculer_cout(voisin, matrice_distances)
            if (cout_voisin < cout_S_prime and
                (voisin not in liste_tabou or cout_voisin < meilleur_cout)):
                S_prime = voisin
                cout_S_prime = cout_voisin

        # Mise à jour de la meilleure solution trouvée
        if S_prime and cout_S_prime < meilleur_cout:
            meilleure_solution = S_prime[:]
            meilleur_cout = cout_S_prime
            if stop_stagnation:
              iter_sans_amelioration = 0  # Réinitialisation du compteur d'itérations sans amélioration
        else:
            if stop_stagnation:
              iter_sans_amelioration += 1  # Incrémenter le compteur

        # Mise à jour de la solution actuelle et de la liste tabou
        S = S_prime or S
        liste_tabou.append(S)
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)

        # Intensification: Revenir à la meilleure solution périodiquement
        if intensifier:
          if iteration % 100 == 0 and iteration != 0:
              S = meilleure_solution

        # Diversification: Relancer avec une nouvelle solution initiale si stagnation
        if diversifier:
          if iteration % 200 == 0 and iteration != 0:
              S = generer_solution_initiale(nb_villes)
              liste_tabou = []  # Réinitialiser la liste tabou après la diversification

        # Stop si stagnation
        if stop_stagnation:
          if iter_sans_amelioration >= max_iter_sans_amelioration:
              print(f"Stagnation atteinte à l'itération {iteration}, arrêt prématuré.")
              break

        # Variation de la taille de la liste tabou
        taille_liste_tabou = 10 if iteration % 15 == 0 else 15

        print(f"Iteration {iteration}: Cout actuel = {cout_S_prime}, Meilleur cout = {meilleur_cout}")

    return meilleure_solution, meilleur_cout
G=build_graph_from_distance_matrix(matrice_distances)
# Paramètres de l'algorithme
nb_villes = 24  # Nombre de villes
nb_iterations_max = 1000  # Nombre d'itérations maximal
taille_liste_tabou_initiale = 20  # Taille initiale de la liste tabou
diversifier = True
intensifier = True
stop_stagnation = False
max_iter_sans_amelioration =30
initialize_randomly = False
# Exécution de l'algorithme de recherche tabou
meilleure_solution, cout_meilleure_solution = recherche_tabou(G,nb_villes, nb_iterations_max, taille_liste_tabou_initiale, matrice_distances, diversifier, intensifier, stop_stagnation, 30, initialize_randomly)

